In [22]:
import pandas as pd
from collections import defaultdict
import dvc.api
from tqdm import tqdm

In [23]:
df_original = pd.read_csv(dvc.api.get_url('datasets/data/wish_attr_extract_label/appen/input_batch_processed/appen_product_attribution_batch1.csv', 
    repo='git@github.com:ContextLogic/multitask-llm-rnd.git'))

In [24]:
df = pd.read_csv('appen/output_batch/Product Attribution_02.03.23.csv')

In [25]:
df.head(2)

,Label Ordering,Sample Method,Product ID,Title,Product Description,Image URL,Product Category,L2 Category,Rater 1 Answer,Rater 2 Answer,Final Answer
0,0,only_text,611bbb365b0bd8698b670d9d,"Rattan Basket Pet Dome and Animal Bed, with Me...",Features\n- Rattan wicker brings a classic sty...,NaN,Home & Garden > Pet Products > Cat Supplies > ...,Home & Garden > Pet Products,[''],['Home & Garden > Pet Products > Cat Supplies ...,['Home & Garden > Pet Products > Cat Supplies ...
1,1,only_text,61a9c0160dcaf9e1da138df1,10Pcs 7Pcs 2Pcs 1Pcs Tempered Glass For Iphone...,welcome to my store!\n\nFeatures:\nCompatible ...,NaN,Cellphones & Telecommunications > Mobile Phone...,Cellphones & Telecommunications > Mobile Phone...,['Cellphones & Telecommunications > Mobile Pho...,['Cellphones & Telecommunications > Mobile Pho...,['Cellphones & Telecommunications > Mobile Pho...


In [26]:
df_original.head(2)

,category_path,product_id,title,product_description,main_image_url,sample_method,label_ordering
0,Home & Garden > Pet Products > Cat Supplies > ...,611bbb365b0bd8698b670d9d,"Rattan Basket Pet Dome and Animal Bed, with Me...",Features\n- Rattan wicker brings a classic sty...,NaN,only_text,0.0
1,Cellphones & Telecommunications > Mobile Phone...,61a9c0160dcaf9e1da138df1,10Pcs 7Pcs 2Pcs 1Pcs Tempered Glass For Iphone...,welcome to my store!\r\n\r\nFeatures:\r\nCompa...,NaN,only_text,1.0


In [27]:
df_original['label_ordering'].isna().mean()

9.746588693957115e-05

In [28]:
df_original = df_original[~df_original['label_ordering'].isna()]

In [29]:
df_original['label_ordering'] = df_original['label_ordering'].astype(int)

In [30]:
len(df)

20544

In [31]:
df = df.merge(df_original[['label_ordering', 'title', 'product_description', 'main_image_url', 'category_path', 'product_id', 'sample_method']].rename(columns={
    'label_ordering': 'Label Ordering',
    'title': 'title_original', 
    'product_description': 'product_description_original', 
    'main_image_url': 'main_image_url_original', 
    'category_path': 'category_path_original', 
    'product_id': 'product_id_original', 
    'sample_method': 'sample_method_original'
}), on='Label Ordering', how='inner')

In [32]:
(df['Title'] != df['title_original']).mean()

0.07296534267912773

In [33]:
df[df['Title'] != df['title_original']].sample(2)[['Title', 'title_original']].to_dict('records')

[{'Title': "Natural Green Chalcedony Pendant Fashion Jewelry Men and Women's 925 Inlaid Agate Necklace Jewelry Wholesale",
  'title_original': "Natural Green Chalcedony Pendant Fashion Jewelry Men and Women's 925 Inlaid Agate Necklace Jewelry Wholesale  "},
 {'Title': '100g Mineral Stone Attractive Irregular  Practical Purple Natural Stone Faux Amethyst Room Decoration',
  'title_original': ' 100g Mineral Stone Attractive Irregular  Practical Purple Natural Stone Faux Amethyst Room Decoration    '}]

In [34]:
(df['Product Category'] != df['category_path_original']).mean()

0.0

In [37]:
errors = []
error_c = 0
for i in tqdm(df[['Label Ordering', 'Final Answer', 'Product Category']].to_dict('records')):
    for j in eval(i['Final Answer']):
        if not j.startswith(i['Product Category']):
            error_c += 1
            errors.append(i)
            break
error_c / len(df)

100%|██████████| 20544/20544 [00:00<00:00, 52972.25it/s]


0.2707359813084112

In [38]:
# product
errors[:2]

[{'Label Ordering': 2,
  'Final Answer': "['Home & Garden > Arts, Crafts & Sewing > Apparel Sewing & Fabric > Adhesive Fastener Tape > Materials > Canvas']",
  'Product Category': 'Home & Garden > Arts, Crafts & Sewing > Needle Arts & Crafts > Diamond Painting Cross-Stitch'},
 {'Label Ordering': 3,
  'Final Answer': '[\'Shoes > Men\\\'s Shoes > Men\\\'s Sandals > Department > Women"\']',
  'Product Category': "Shoes > Women's Shoes > Women's Sandals"}]

In [39]:
df[df['title_original'] != df['Title']][['Label Ordering', 'Title', 'title_original']].sample(2).to_dict('records')

[{'Label Ordering': 11558,
  'Title': "ProPlus Chandelles en aluminium pour remorques et caravanes Jusqu\\'√É∆í&nbsp; 3600 kg",
  'title_original': "ProPlus Chandelles en aluminium pour remorques et caravanes Jusqu\\'Ã&nbsp; 3600 kg"},
 {'Label Ordering': 16425,
  'Title': '1 Set  Sand Shovels Non-slip Toilet Cleaning    Cat Supplies',
  'title_original': ' 1 Set  Sand Shovels Non-slip Toilet Cleaning    Cat Supplies  '}]